# Data Structures and Algorithms in Python - Ch.9: Priority Queues and Heaps
### AJ Zerouali, 2023/10/05

## 0) Introduction

**References:**

- Chapter 9 of "Data structures and algorithms in Python", by Goodrich, Tamassia and Goldwasser (primary, abbreviated [GTG13]). 
- Section 16 of "Python for Data Structures, Algorithms, and Interviews!" by Jose Portilla (to a much lesser extent).

**Comments:**
- A very important part this chapter is section 9.4 of [GTG13], which covers selection-sort, insertion-sort and heap-sort.

## 1) Priority queues

This is form section 9.1 of [GTG13]. Some applications are explained here: https://en.wikipedia.org/wiki/Priority_queue#Applications.

A **priority queue** is an upgrade to a queue where the elements are not ordered on the LIFO basis, but rather depending on the priority of the element. Typically, the priority is determined by a **key** here, in the sense that any element added to the priority queue consists of a couple **(key, value)**. The purpose of this data structure is to allow arbitrary insertions, and the next element removed from this container is the one with the minimum key.

The interface of priority queue object *P* will typically have the following methods:
- *P.add(k,v):* Adds an element *(k,v)* where *k* is the key and *v* is the value.
- *P.min():* Returns *(k,v)*, where *k* is the minimal key in *P*.
- *P.remove_min():* Remove the item with minimal key from *P*.
- *P.is_empty():* Checks if *P* is empty.
- *len(P):* Returns the number of elements stored in *P*.

Some remarks:
- The keys of the items stored remain fixed.
- If the keys aren't unique, then *min()* and *remove_min()* report arbitrary choices.

**Comments on the implementation:**
- To implement the elements of a priority queue, [GTG13] discuss the **composition design pattern**, initially introduced in section 7.6 (p.287). The idea is that one instance is composed of several objects. This is the content of section 9.2.1.
- The remainder of 9.2 discusses two ways of implementing priority queues. As usual, the authors use a positional (doubly-linked) list as an underlying container for the key-value pairs. Said ilst can be either unsorted or sorted.
- The unsorted list implementation is discussed in 9.2.2 (see code on p.367). The main facts to note here are that the addition of an item will run in $O(n)$ time, while the *min()* and *remove_min()* methods will run in $O(n)$ time.
- The sorted list implementation is discussed in 9.2.3 (code on p.369). The difference here is that the elements of the priority queue are added after checking the order, meaning that *add()* will run in $O(n)$ time. In contrast, the methods *min()* and *remove_min()* will run in $O(1)$ time when this implementation is used.

**Comment (23/10/02):** I am skipping these implementations for now. My priority is the heap implementation of the priority queue, and heaps are implemented using arrays (see also lecture 112 of Portilla's DSA course). My end-goal are the sorting algorithms.

In view of the remarks above, there is a more efficient way of implementing priority lists that is based on binary heaps, which are a special class of trees.

## 2) Heaps

### 2.a - Basics of heaps
This section is based on sec.9.3.1 of [GTG13].

#### Definition 
A (binary) **heap** is a binary tree $T$ whose nodes/positions store key-value couples, and satisfying  two additional conditions:
1) **The heap-order property:** For every position $p$ in the heap $T$, the key stored at $p$ is greater than the key stored at $p$'s parent.
2) **Complete binary tree property:** A binary tree $T$ is complete if every level $i=0,1,\cdots,h-1$ contains the maximal number of nodes possible $2^i$, and the remaining nodes at level $h$ reside at the lefmost possible positions at that level (see [GTG13, Fig.9.1] for a depiction).

The height of a heap satisfies an important property:

#### Proposition 9.2:
A heap $T$ storing $n$ entries has height $h=\lfloor \log(n)\rfloor$.

#### Proof sketch:
For a binary tree, we know that $h\ge \log(n+1)-1$. Since $T$ is complete, the number of internal nodes is $2^h-1$, with at least one node in level $h$, meaning that $n\ge 2^h$. Thus $\log(n+1)-1\le h\le \log(n)$. Since the difference between the bounds is lower than $1$ and $h$ is an integer, we must have that $h=\lfloor \log(n)\rfloor$.

### 2.b - Practical considerations

The reference for this part is [GTG13, Sec.9.3.2].

#### Level numbering as indices

For the sake of simplicity, we will implement our heap using an array in the next section, and use this class to realize a priority queue. Furthermore, we will adopt the convention by which the highest priority element has the minimal key, which translates into using a min-heap, for which the root is the minimal element. Instead of using the abstract position class of [GTG13], the nodes of our heap will be indexed by the *level numbering function* [GTG13, Sec.8.3.2], which is defined as follows:
- If the position $p$ is the root, then: $f(p)=0$.
- If $p$ is the left child of a position $q$, then: $f(p)=2f(q)+1$.
- If $p$ is the right child of a position $q$, then: $f(p)=2f(q)+2$.

When the heap nodes are stored in in an array, the level numbering function gives us an easy way of accessing the children and the parent of a node. There are other advantages of this approach that we will discuss later, and it is worth noting here that in Portilla's course, the level numbering function differs from this one by one (he keeps position 0 empty).

The heavy lifting of this implementation lies in how we add a node to the heap and how we remove the minimal element. Again, we want our priority queue to have the following public interface:
- *P.add(k,v):* Adds an element *(k,v)* where *k* is the key and *v* is the value.
- *P.min():* Returns *(k,v)*, where *k* is the minimal key in *P*.
- *P.remove_min():* Remove the item with minimal key from *P*.
- *P.is_empty():* Checks if *P* is empty.
- *len(P):* Returns the number of elements stored in *P*.

We next discuss some details for the methods *add()* and *remove_min()*.

#### Adding an element

One of the advantages of using an array-based implementation for our heap is that appending a new node to the array will preserve the **complete binary tree property**. Suppose that the heap has height $h$. Intuitively, the appension of a new node will either:
- Add it to the position just after the rightmost node occupying level $h$, if this level is not yet full.
- Add it to the leftmost position of a new level $h+1$, if level $h$ is full.

Thus, as we add nodes to an array-based heap, we do not need to explicitly reshape this tree when using the level numbering. The issue we need to deal with however is the **heap-order property**, as we need to ensure that the newly added node is in the correct level of the tree, and that its key is lower than that of its parent node. Obviously, if the newly added node $(k_n,v_n)$ has parent $(k_p, v_p)$, and if the keys are such that $k_p>k_n$, then we will have to move $(k_n,v_n)$ to a position of lower level in the heap.

Since we potentially have to move the new node to a lower level in several iterations, we see that verifying all the keys in the base array leads to an execution bottleneck. Another advantage of using a heap becomes clear at this point: By checking only the keys of the newly added node and its parent, we are reducing the time complexity of this addition method. In practice, we just have to use a while loop, and swap the positions of new node and its parent in the heap array as long as $k_p>k_n$. This method is called **up-heap bubbling**, and a good depiction is given in [GTG13, Fig.9.2].

#### Removing the minimal element

With the implementation discussed in this section, it is obvious that the minimal element in the heap is its root. This means that naively, *remove_min()* will simply return the root of the tree after removing it. The issue now is that when removing first element from the heap array and simply shifting all the entries by $-1$, we are not necessarily left with a heap anymore. Visually, just removing the root creates two trees, and we need a *remove_min()* algorithm that reorganizes the array into a heap.

The practical solution is as follows:
1) First, to preserve the **complete binary tree** property, we interchange the last position in the tree with the root, then remove and return the last entry of the base array. With this approach, we have removed the minimal element, and all the nodes in the highest level remain in all the leftmost positions.
2) Second, iteratively move the new root node down the tree to preserve the **heap-order property**. If $p$ denotes the (new) node positioned at the root, define $c$ such that:
    * If $p$ has no right child, then $c$ is $p$'s left child.
    * If $p$ has two children, let $c$ be the child of lowest key.
    
    Proceeding recursively while $k_p>k_c$, swap the entries at nodes $p$ and $c$.
    This process is called **down-heap bubbling**.
    
The down-heap bubbling algorithm is depicted in [GTG13, Fig.9.3]. An important remark here is how the child $c$ is defined in (2) above. This choice guarantees that before swapping, the node $c$ has the smallest key.

#### Time complexity of the priority queue's methods

At first glance, using up/down-heap bubbling seems like a time intensive process. It is instructive to note that in the worst case, these sorting procedures take $h$ steps where $h$ is the height of the tree, and by Proposition 9.2, they are in fact $O(\log n)$ (amortized, when using an array-based implementation) if the heap has $n$ nodes.

Below, we will implement our heap so that the priority queue's *is_empty()* and *len()* are $O(1)$. The last method is *min()*, which is clearly $O(1)$ since it returns the first entry of the base array.

### 2.c - Python's *heapq* module

The official documentation for this module can be found here:

https://docs.python.org/3/library/heapq.html

This module does not implement any heap or priority queue classes, but it provides functions that allow one to treat a standard *list* as a heap. Given a list *L*, this module provides the following functions:
* *heapify(L)*: Transforms an arbitrary list *L* to satisfy the heap-order property. Executes the **bottom-up heap construction** algorithm [GTG13, Sec.9.3.6] in $O(n)$ time.
* nlargest(k, L): L could be any iterable. Returns a list of *k* largest values from *L*.
* nsmallest(k, L): L could be any iterable. Returns a list of *k* smallest values from *L*.

The next functions presume that the list *L* satisfies the heap-order property:
* *heappush(L,e)*: Push the element e into *L* and restore the heap-order property. Implements the *add()* method discussed above and executes in $O(\log n)$ time.
* *heappop(L)*: Pop and return the smallest value in *L*. Implements the *remove_min()* method discussed above and executes in $O(\log n)$ time.
* *heappushpop(L,e)*: Combines the previous methods more efficiently than separate calls, by pushing and then popping.
* *heapreplace(L,e)*: Similar to *heappushpop()* but pops the minimal element of *L* before pushing *e*.

We can illustrate this with nodes given by 2-tuples where the key is a string and the value is an integer. Here is an example:

In [1]:
# Import the heapq module
import heapq
# Initialize a list
L = [("K", 28), ("X",4), ("M",7), ("Z",39), ("L",5), ("T",37), ("A",100)]

In [2]:
# Transform list into array-based heap
heapq.heapify(L)
# Show heapified list
print(L)

[('A', 100), ('L', 5), ('K', 28), ('Z', 39), ('X', 4), ('T', 37), ('M', 7)]


In [3]:
# Remove minimal element from the heap L
m = heapq.heappop(L)
# Display results:
print(f"Previous root of L: {m}")
print(f"Current heap L: L = {L}")

Previous root of L: ('A', 100)
Current heap L: L = [('K', 28), ('L', 5), ('M', 7), ('Z', 39), ('X', 4), ('T', 37)]


In [4]:
# Add a new node to the heap L:
heapq.heappush(L, ("B", 800))
print(f"Current heap L: L = {L}")

Current heap L: L = [('B', 800), ('L', 5), ('K', 28), ('Z', 39), ('X', 4), ('T', 37), ('M', 7)]


## 3) Implementing a priority queue with an array-based heap

### 3.a - Some implementation notes

Following [GTG13, Sec.9.3.4], we implement a priority queue as an array-based heap. There are 3 parts to this implementation:

#### Public methods of heap priority queue class:

The public methods will be precisely the methods described in the first section, namely:
- *P.add(k,v):* Adds an element *(k,v)* where *k* is the key and *v* is the value.
- *P.min():* Returns *(k,v)*, where *k* is the minimal key in *P*.
- *P.remove_min():* Remove the item with minimal key from *P*.
- *P.is_empty():* Checks if *P* is empty.
- *len(P):* Returns the number of elements stored in *P*.

It is worth noting that our priority queue will have only one attribute: the (private) *data* list that will store the nodes. The size of the heap will obviously be the length of this list.

#### Abstract base priority queue class

To simplify matters, we use the abstract base class implemented in [GTG13, Sec.9.2.1] (see https://github.com/mjwestcott/Goodrich/blob/master/ch09/priority_queue_base.py).

The most notable point here is the use of a private class *item* for the key-value objects.

#### Private methods of heap priority queue class:

This is where the trickiest implementations will be done. The core methods of this class are the implementations of the up-heap and down-heap bubbling algorithms to respectively add a new node and remove the root. Obviously, these methods will need to access the parent, left child, and right child of a given position in the base array, as well as swap two given nodes. We will therefore need the following additional private methods:

* *_parent(i)*: Returns the index of the parent of the node at index *i*.
* *_left_child(i)*: Returns the index of the left child of the node at index *i*.
* *_right_child(i)*: Returns the index of the right child of the node at index *i*.
* *_has_left(i)*: Checks if node at index *i* has a left child.
* *_has_right(i)*: Checks if node at index *i* has a right child.
* *_swap(i,j)*: Swaps the nodes at positions *i* and *j*.
* *_bubble_up(i)*: Execute up-heap bubbling, called by *add()*.
* *_bubble_down(i)*: Execute down-heap bubbling, called by *remove_min()*.



### 2.b - Abstract priority queue base class

In [1]:
class PriorityQueueBase:
    """Abstract base class for a priority queue."""

    #------------------------------ nested _Item class ------------------------------
    class _Item:
        """Lightweight composite to store priority queue items."""
        
        __slots__ = '_key', '_value'
        
        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key    # compare items based on their keys

        def __repr__(self):
            #return '({0},{1})'.format(self._key, self._value) #Original
            return f"({self._key}, {self._value})"

    #------------------------------ public behaviors ------------------------------
    def is_empty(self):                  # concrete method assuming abstract len
        """Return True if the priority queue is empty."""
        return len(self) == 0
    
    def __len__(self):
        """Return the number of items in the priority queue."""
        raise NotImplementedError('must be implemented by subclass')
    
    def add(self, key, value):
        """Add a key-value pair."""
        raise NotImplementedError('must be implemented by subclass')
        
    def min(self):
        """
            Return but do not remove (k,v) tuple with minimum key.
            Raise ValueError if empty.
        """
        raise NotImplementedError('must be implemented by subclass')
        
    def remove_min(self):
        """
            Remove and return (k,v) tuple with minimum key.
            Raise Empty exception if empty.
        """
        raise NotImplementedError('must be implemented by subclass')

### 2.c - Main class:

In [2]:
class HeapPriorityQ(PriorityQueueBase):
    '''
        Implementation of a priotiy queue 
        as an array-based heap.
    '''
    #---------- Constructor ----------
    def __init__(self):
        '''
            Constructor for priority queue
        '''
        self._data = []
        
    #---------- Private methods ----------
    
    def _parent(self, i):
        '''
            Return index of parent of node at position i.
            (Should not be called for the root)
        '''
        return (i-1)//2
    
    def _left_child(self,i):
        '''
            Return index of left child of node at position i.
        '''
        return 2*i+1
    
    def _has_left(self, i):
        '''
            Check if node at index i has a left child.
        '''
        return self._left_child(i)<len(self._data)
    
    def _right_child(self,i):
        '''
            Return index of right child of node at position i.
        '''
        return 2*(i+1)
    
    def _has_right(self, i):
        '''
            Check if current index has a left child.
        '''
        return self._right_child(i)<len(self._data)
    
    def _swap(self, i, j):
        '''
            Interchange items in nodes at positions
            i and j.
        '''
        self._data[i], self._data[j] = self._data[j], self._data[i]
        
    def _bubble_up(self, i):
        '''
            Execute up-heap bubbling on node
            at position i.
        '''
        # Check that i is not the root
        if i>0:
            j = i
            p = self._parent(j)
            
            # Core loop
            while self._data[j]<self._data[p] and j>0:
                self._swap(j,p)
                j, p = p, self._parent(p)
    
    def _bubble_down(self, i):
        '''
            Execute down-heap bubbling on node
            at position i.
        '''
        # Check that node has a left child at least
        ## If False, then current index is in highest level of heap
        if self._has_left(i):
            lchild = self._left_child(i)
            min_child = lchild
            
            # Check if node has right child and if lower than left child
            if self._has_right(i):
                rchild = self._right_child(i)
                if self._data[rchild]<self._data[lchild]:
                    min_child = rchild
            
            # Check if i and min_child need swap
            if self._data[min_child]<self._data[i]:
                self._swap(min_child, i)
                self._bubble_down(min_child)
            
        
    
    #---------- Public methods ----------
    def __len__(self):
        '''
            Return number of elements stored in the priority queue
        '''
        return len(self._data)
    
    def min(self):
        '''
            Return (key, value) tuple corresonding to minimal element 
            of the priority queue.
            Raises ValueError if the queue is empty.
        '''
        if self.is_empty():
            raise ValueError("Cannot return min element of an empty priority queue")
        
        item = self._data[0]
        return (item._key, item._value)
    
    def add(self, k,v):
        '''
            Add key-value item (k,v) to the priority queue,
            in such a way that the complete binary tree
            and the heap-order properties are satisfied
        '''
        # Append new item to base array
        self._data.append(self._Item(k,v))
        
        # Execute up-heap bubbling
        self._bubble_up(len(self._data)-1)
        
    
    def remove_min(self):
        '''
            Remove and return minimal element of the priority
            queue, and reshape heap.
            Raises ValueError if tree is empty
        '''
        # Raise error if heap is empty
        if self.is_empty():
            raise ValueError("Cannot remove minimal element of an empty priority queue")
            
        # Swap root and last entry of heap
        self._swap(0, len(self._data)-1)
        min_element = self._data.pop()
        
        # Execute down-heap bubbling
        self._bubble_down(0)
        
        # Output
        return (min_element._key, min_element._value)

**Comments:**
In [GTG13], the *_bubble_up()* method is implemented as a recursion, but I elected to use a while loop. For the *_bubble_down()* method, since we need to check the left and right child at each iteration, the recursion was more optimal.

### 2.c - An example:

We verify our implementation on the example depicted in figures 9.1-9.3 of [GTG13, Sec.9.3].

First, we reproduce the heap in Figure 9.1. It is easy to see from the heap order that we have built the same binary tree:

In [6]:
from random import shuffle
import heapq
from copy import deepcopy
'''
L_ = [(6,"Z"), (20,"B"), (13, "W"), (7, "Q"), (11, "S"), (15,"K"),
     (5, "A"), (9, "F"), (25, "J"), (12, "H"), (16,"X"), (14,"E"), (4,"C")
    ]
'''
L = [(4,"C"), (5, "A"), (6,"Z"), (15,"K"), (9, "F"), 
      (7, "Q"), (20,"B"), (16,"X"), (25, "J"), (14,"E"), 
      (12, "H"), (11, "S"), (13, "W"), 
     ]

In [5]:
# Build heap of Fig.9.1
heap1 = HeapPriorityQ()
for e in L:
    heap1.add(e[0], e[1])

# Display resulting tree
heap1._data

[(4, C),
 (5, A),
 (6, Z),
 (15, K),
 (9, F),
 (7, Q),
 (20, B),
 (16, X),
 (25, J),
 (14, E),
 (12, H),
 (11, S),
 (13, W)]

Now we reconstruct the tree of Fig.9.1. We copy the heap above, and add the element *(2, "T")*:

In [7]:
# Copy base heap
heap2 = deepcopy(heap1)

# Add element in Fig.9.2
heap2.add(2,"T")

# Display base array
heap2._data

[(2, T),
 (5, A),
 (4, C),
 (15, K),
 (9, F),
 (7, Q),
 (6, Z),
 (16, X),
 (25, J),
 (14, E),
 (12, H),
 (11, S),
 (13, W),
 (20, B)]

Next, we redo the Fig.9.3 example, by removing the root of the first example:

In [8]:
# Copy base heap
heap3 = deepcopy(heap1)

# Add element in Fig.9.2
prev_root = heap3.remove_min()

# Print previous root
print(f"Min. element of heap1: prev_root = {prev_root}")

# Display base array
heap3._data

Min. element of heap1: prev_root = (4, 'C')


[(5, A),
 (9, F),
 (6, Z),
 (15, K),
 (12, H),
 (7, Q),
 (20, B),
 (16, X),
 (25, J),
 (14, E),
 (13, W),
 (11, S)]